In [7]:
import pandas as pd
import numpy as np
import random
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import timeDistance
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list
from ipynb.fs.defs.functions import check_cities
from ipynb.fs.defs.functions import random_schedule_day
from ipynb.fs.defs.functions import translate_name

dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

base1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
base2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")

In [25]:
fur1 = translate_name(dft, pd.read_csv("../Dati/turni_f/original_start/f1_original.csv", delimiter=";", encoding = "ISO-8859-1"))
fur2 = translate_name(dft, pd.read_csv("../Dati/turni_f/original_start/f2_original.csv", delimiter=";", encoding = "ISO-8859-1"))

random_f1 = pd.DataFrame(data={})
random_f2 = pd.DataFrame(data={})

for d in fur1.columns:
    sample = random.sample(range(1,len(fur1[d].dropna())-1), len(fur1[d].dropna())-2)
    center = [fur1[d][0]] + list(fur1[d][sample]) + [list(fur1[d].dropna())[-1]]
    random_f1 = pd.concat([random_f1, pd.Series(center).rename(d)], axis=1)

for d in fur2.columns:
    sample = random.sample(range(1,len(fur2[d].dropna())-1), len(fur2[d].dropna())-2)
    center = [fur2[d][0]] + list(fur2[d][sample]) + [list(fur2[d].dropna())[-1]]
    random_f2 = pd.concat([random_f2, pd.Series(center).rename(d)], axis=1)

,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Offanengo, Clavelli Martini, 5","Castelverde, Ubaldo Ferrari, 15","Casalmaggiore, Guglielmo Marconi, 8"
2,"Persico Dosimo, Albert ,7","Cremona, Palosca n. 2","Offanengo, Clavelli Martini, 5"
3,"Vailate, Dante Alighieri, 7","Casalbuttano ed Uniti, Jacini, 23","Pizzighettone, Giuseppe Garibaldi, 18"
4,"Gadesco-Pieve Delmona, Giovanni Lonati 9","San Giovanni in Croce, Feudatari, 25 A","Crema, Civerchi, 9"
5,"Crema, Civerchi, 9",NaN,"Casalpusterlengo, Pusterla 7"
6,"Casalpusterlengo, Pusterla 7",NaN,"San Giovanni in Croce, Feudatari, 25 A"
7,"Pescarolo ed Uniti, Giuseppe Mazzini, 73",NaN,"Izano, Gerardo da Izano, 4/C"
8,"Ostiano, Mazzini, 1",NaN,"Pizzighettone, Giuseppe Garibaldi, 18"
9,"Cremona, Felice Cavallotti, 2",NaN,"Cremona, Sesto 39"


In [26]:
random_f1.to_csv("../Dati/turni_f/random_inday/random_day_f1.csv", sep=";", encoding = "ISO-8859-1")
random_f2.to_csv("../Dati/turni_f/random_inday/random_day_f2.csv", sep=";", encoding = "ISO-8859-1")

In [136]:
occurrences = pd.Series(index=dft.columns, dtype=int)
tot = []
for d in base1.columns:
    for city in base1[d].dropna()[1:len(base1[d].dropna())-1]:
        occurrences[city] += 1
        tot.append(city)
for d in base2.columns:
    for city in base2[d].dropna()[1:len(base2[d].dropna())-1]:
        occurrences[city] += 1
        tot.append(city)
   

In [ ]:
cardinality = [8,16,16,16,8]

random_df = pd.DataFrame(data={})
for d in base1.columns:
    i = list(base1.columns).index(d)
    
    c = cardinality[i]
    cur_route = random.sample(range(0,len(occurrences)), c)
    daily_cities = []
    for el in cur_route:
        daily_cities.append(occurrences.index[el])
    for el in cur_route:
        occurrences[occurrences.index[el]] -=1
    to_remove = []
    for el in cur_route:
        if occurrences[occurrences.index[el]] == 0:
            to_remove.append(occurrences.index[el])
    occurrences = occurrences.drop(labels=to_remove)
    
    random_df = pd.concat([random_df, pd.Series(daily_cities).rename(d)], axis=1)

random_df

In [211]:
leg = 1
counter = 100
while(leg!=0 and counter>0):
    leg = 0
    occurrences = pd.Series(index=dft.columns, dtype=int)
    tot = []
    for d in base1.columns:
        for city in base1[d].dropna()[1:len(base1[d].dropna())-1]:
            occurrences[city] += 1
            tot.append(city)
    for d in base2.columns:
        for city in base2[d].dropna()[1:len(base2[d].dropna())-1]:
            occurrences[city] += 1
            tot.append(city)

    cardinality = [12,22,22,22,11]
    random_df = pd.DataFrame(data={})
    for d in base1.columns:
        i = list(base1.columns).index(d)
        c = cardinality[i]

        cur_route = random.sample(range(0,len(tot)), c)
        daily_cities = []
        for el in cur_route:
            daily_cities.append(tot[el])
        for el in daily_cities:
            tot.remove(el)

        random_df = pd.concat([random_df, pd.Series(daily_cities).rename(d)], axis=1)

    
    for d in random_df.columns:
        curr = list(random_df[d].dropna())
        leg += len(set([(i, curr.count(i)) for i in curr if curr.count(i)>1]))
    print(counter)
    counter -= 1
    

100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [216]:
random_df = possibile
for d in random_df.columns:
        curr = list(random_df[d].dropna())
        print(set([(i, curr.count(i)) for i in curr if curr.count(i)>1]))

set()
{('Casalmaggiore, Guglielmo Marconi, 8', 2)}
{('Crema, Civerchi, 9', 2)}
set()
set()


In [218]:
random_df.to_csv("../Dati/turni_f/random_schedule/random_complete.csv", sep=";", encoding = "ISO-8859-1")